### Project 6 - Exceptions

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

Furthermore we have determined that we will need the following categories of exceptions:

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

##### Bonus

Log the traceback too - you'll have to do a bit of research for that! 

I'm going to use the `TracebackException` class in the `traceback` module:

https://docs.python.org/3/library/traceback.html#tracebackexception-objects

In particular, look at the class method `from_exception` (and remember that inside your exception class, the exception will be `self`!) and the `format` instance method. That method returns a generator, so you'll need to `list` it to print out everything in that traceback.

Good luck!

### Solution

In [1]:
from http import HTTPStatus
from datetime import datetime
import json
import traceback

In [2]:
class WidgetException(Exception):
    '''Exception for widget'''
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    user_err_message = 'This is message for user'
    internal_err_message = 'Internal message'
    
    def __init__(self, *args, user_err_message=None):
        if args:
            self.internal_err_message = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_message)
        
        if user_err_message is not None:
            self.user_err_message = user_err_message
            
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()
    
    def log(self):
        exception = {
            'type': type(self).__name__,
            'http_status': self.http_status,
            'message': self.internal_err_message,
            'args': self.args[1:]
        }
        print(f'EXCEPTION {datetime.utcnow().isoformat()}: {exception}')
    
    def to_json(self):
        err = {
            'status': self.http_status,
            'message': self.user_err_message,
            'category': type(self).__name__
        }
        return json.dumps(err)

In [3]:
class SuplierException(WidgetException):
    '''Suplier exception'''
    internal_err_message = 'Supplier exception.'
    
class NotManufacturedException(SuplierException):
    '''Not manufactured anymore'''
    internal_err_message = 'Widget is no longer manufactured by supplier.'

class ProductionDelayedException(SuplierException):
    '''Production delayed'''
    internal_err_message = 'Widget production has been delayed by supplier.'

class ShippingDelayedException(SuplierException):
    '''Shipping delayed'''
    internal_err_message = 'Widget shipping has been delayed by supplier.'

In [4]:
class CheckoutExceptions(WidgetException):
    '''Checkout exceptions'''
    internal_err_message = 'Checkout exception.'

class InventoryTypeExceptions(CheckoutExceptions):
    '''Inventory Type Exceptions'''
    internal_err_message = 'Checkout inventory exception.'

class OutOfStockExceptions(InventoryTypeExceptions):
    '''out of stock'''
    internal_err_message = 'Inventory out of stock'

class PricingExceptions(CheckoutExceptions):
    '''pricing exceptions'''
    internal_err_message = 'Checkout pricing exception.'

class InvalidCodeExceptions(PricingExceptions):
    '''invalud code coupon'''
    internal_err_message = 'Invalid checkout coupon code.'
    http_status = HTTPStatus.BAD_REQUEST

class StackCouponExceptions(PricingExceptions):
    '''cannot stack coupons'''
    internal_err_message = 'Cannot stack checkout coupon codes.'
    http_status = HTTPStatus.BAD_REQUEST

In [7]:
try:
    raise InvalidCodeExceptions(user_err_message='Sorry this is coupon now is unavailable')
except InvalidCodeExceptions as ex:
    ex.log()
    print('-------------')
    print(ex.to_json())
    print('-------------')
    print(''.join(ex.traceback))

EXCEPTION 2020-07-17T09:56:44.316676: {'type': 'InvalidCodeExceptions', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Invalid checkout coupon code.', 'args': ()}
-------------
{"status": 400, "message": "Sorry this is coupon now is unavailable", "category": "InvalidCodeExceptions"}
-------------
Traceback (most recent call last):
  File "<ipython-input-7-c368b633edbf>", line 2, in <module>
    raise InvalidCodeExceptions(user_err_message='Sorry this is coupon now is unavailable')
InvalidCodeExceptions: Invalid checkout coupon code.

